## Function fitter using tensroflow fitter
Test and demo of curve fitting using wrapper tensoflow. For the moment only ADAm optimizer supported.
* user defined function to mnimize
* user defined weight function
* covaraince matrix extraction
* bootstrap method 

### Example functions:
* linear
* non linear: sinus, exponential

### Content:
1. Define input paraemeter of test (number of points and number of test fits)
2. Define test functions (linear, sinus, exp) as tensorflow functions
3. Define loss function
4. Single fit example
  *  4.a Create input data
  *  4.b Make a fit
5. Test of the fitters
   * n tests data stes with npoints to fit
   * Test statistical properties of exctρacted data
   * distribution of residual Δ, Δ/σ to test quality of fits 

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-2"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp

## 1. define input paraemeter of test (number of points and number of test fits)

In [ ]:
npoints=50
ntest=50
# setting paremeters 
num_step=50
learning_rate=0.4
epsilon=0.0001

## 2. create different test functions as tensorflow functions

In [ ]:
def testfunc_sin(x, a, b, c):
    a=tf.cast(a,tf.float64)
    b=tf.cast(b,tf.float64)
    c=tf.cast(c,tf.float64)
    return a*tf.sin(b*x)+c

def testfunc_lin(x, a, b):
    a=tf.cast(a,tf.float64)
    b=tf.cast(b,tf.float64)
    return a*x+b

def testfunc_exp(x, a, b):
    a=tf.cast(a,tf.float64)
    b=tf.cast(b,tf.float64)
    return tf.exp(a*x)+b

##  3.)  Define  loss function
* could be any function f(y_pred, y_true) which returns a  scalar
* By defult we should be able to specify also wieths

In [ ]:
def loss_func(y_pred, y_true):
    return tf.reduce_mean((y_pred - y_true )**2)

def loss_func_w(y_pred, y_true, weights):
    return tf.reduce_mean((y_pred - y_true )**2)+tf.reduce_mean(weights)
 

# 4. Single fit example
  *  4.a Create input data
  *  4.b Make a fit and protin results - compare input and output

In [ ]:
y_vals_sin = []
y_vals_lin = []
y_vals_exp = []
for el in x_vals:
    y_vals_sin.append(testfunc_sin(el, np.random.normal(0.5,0.1), np.random.normal(0.5,0.1), np.random.normal(0.5,0.1)))
    y_vals_lin.append(testfunc_lin(el, np.random.normal(0.5,0.1), np.random.normal(0.5,0.1)))
    y_vals_exp.append(testfunc_exp(el, np.random.normal(0.5,0.1), np.random.normal(0.5,0.1)))
    
weights = np.ones(x_vals.shape[0])   

## 4. b import Fitter and test it for the different functions and normal fitting + BS fitting

In [ ]:
from Fitter import curve_fit, curve_fit_BS, curve_fit_raw

In [ ]:
testfunclist = [testfunc_sin, testfunc_lin, testfunc_exp]
y_vals_list = [y_vals_sin, y_vals_lin, y_vals_exp]
namelist = ["sin","lin","exp"]

In [ ]:
for el in zip(testfunclist,y_vals_list,namelist):
    a,b,c = curve_fit_raw(x_vals, np.array(el[1]), el[0], loss=loss_func)
    a,b,c = curve_fit_raw(x_vals, np.array(el[1]), el[0], loss=loss_func_w, weights = weights)
    a,c = curve_fit(x_vals, np.array(el[1]), el[0], loss=loss_func)
    print()
    print()
    print("function: "+el[2])
    print("parameters:")
    print(a)
    print("covariance matrix:")
    print(c)
    a,c = curve_fit(x_vals, np.array(el[1]), el[0], loss=loss_func_w, weights = weights)
    a,c = curve_fit_BS(x_vals, np.array(el[1]), el[0], loss=loss_func)
    print("parameters BS:")
    print(a)
    print("errors BS:")
    print(c)
    
    a,c = curve_fit_BS(x_vals, np.array(el[1]), el[0], loss=loss_func_w, weights = weights)


# initial parameters should be provided as tf.Variable

In [ ]:
initial_parameters = [tf.Variable(1.),tf.Variable(1.),tf.Variable(1.)]

In [ ]:
a,c = curve_fit(x_vals, np.array(y_vals_sin), testfunc_sin, initial_parameters=initial_parameters)

# use Keras loss

In [ ]:
lhuber = tf.keras.losses.Huber()

In [ ]:
a,c = curve_fit(x_vals, np.array(y_vals_sin), testfunc_sin, loss = lhuber, initial_parameters=initial_parameters)

In [ ]:
a

# test many parameters

In [ ]:
n_tests = 100
y_lin_list = []
y_sin_list = []
y_exp_list = []
for i in range(n_tests):
    y_lin_list.append([])
    y_sin_list.append([])
    y_exp_list.append([])

In [ ]:
a_lin = np.random.uniform(size = n_tests)
b_lin = np.random.uniform(size = n_tests)
sigma_lin = np.random.uniform(size = n_tests)

a_exp = np.random.uniform(size = n_tests)
b_exp = np.random.uniform(size = n_tests)
sigma_exp = np.random.uniform(size = n_tests)

a_sin = np.random.uniform(size = n_tests)
b_sin = np.random.uniform(size = n_tests)
c_sin = np.random.uniform(size = n_tests)
sigma_sin = np.random.uniform(size = n_tests)

In [ ]:
for idx,el in enumerate(zip(y_sin_list, y_lin_list, y_exp_list)):
        for x in x_vals:
            el[0].append(testfunc_sin(x, a_sin[idx], b_sin[idx], c_sin[idx])+np.random.normal(0,sigma_sin[idx]))
            el[1].append(testfunc_lin(x, a_lin[idx], b_lin[idx])+np.random.normal(0,sigma_lin[idx]))
            el[2].append(testfunc_exp(x, a_exp[idx], b_exp[idx])+np.random.normal(0,sigma_exp[idx]))

In [ ]:
pars_out_sin = []
pars_out_lin = []
pars_out_exp = []
errs_out_sin = []
errs_out_lin = []
errs_out_exp = []
pars_out_sinBS = []
pars_out_linBS = []
pars_out_expBS = []
errs_out_sinBS = []
errs_out_linBS = []
errs_out_expBS = []

for i in range(n_tests):
        print(i)
        #a,b = curve_fit(x_vals, np.array(y_sin_list[i]), testfunc_sin)
        #pars_out_sin.append(a)
        #errs_out_sin.append(np.sqrt(np.diag(b)))
        #a,b = curve_fit_BS(x_vals, np.array(y_sin_list[i]), testfunc_sin)
        #pars_out_sinBS.append(a)
        #errs_out_sinBS.append(b)
        a,b = curve_fit(x_vals, np.array(y_lin_list[i]), testfunc_lin,num_step=50,learning_rate=0.4)
        pars_out_lin.append(a)
        errs_out_lin.append(np.sqrt(np.diag(b)))
        a,b = curve_fit_BS(x_vals, np.array(y_lin_list[i]), testfunc_lin,num_step=50,learning_rate=0.4)
        pars_out_linBS.append(a)
        errs_out_linBS.append(b)
        #a,b = curve_fit(x_vals, np.array(y_exp_list[i]), testfunc_exp)
        #pars_out_exp.append(a)
        #errs_out_exp.append(np.sqrt(np.diag(b)))
        #a,b = curve_fit_BS(x_vals, np.array(y_exp_list[i]), testfunc_exp)
        #pars_out_expBS.append(a)
        #errs_out_expBS.append(b)


In [ ]:
#np.array(pars_out_sin)[:,0]

In [ ]:
dictlin = {"a": a_lin, "b": b_lin, "a_pred": np.array(pars_out_lin)[:,0], "b_pred": np.array(pars_out_lin)[:,1],
          "a_err": np.array(errs_out_lin)[:,0], "b_err": np.array(errs_out_lin)[:,1],
          "a_predBS": np.array(pars_out_linBS)[:,0], "b_predBS": np.array(pars_out_linBS)[:,1],
          "a_errBS": np.array(errs_out_linBS)[:,0], "b_errBS": np.array(errs_out_linBS)[:,1]
          }

In [ ]:
#dfsin = pd.DataFrame(dictsin)
dflin = pd.DataFrame(dictlin)
#dfexp = pd.DataFrame(dictexp)

In [ ]:
dflin.head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
histo=plt.hist((dflin["a"]-dflin["a_pred"])/(dflin["a_err"]/np.sqrt(npoints)), bins=50)

In [ ]:
np.mean((dflin["a"]-dflin["a_pred"])/dflin["a_err"]*np.sqrt(npoints-1))

In [ ]:
np.std((dflin["a"]-dflin["a_pred"])/dflin["a_err"]*np.sqrt(npoints-3))